In [23]:
gdf = gpd.read_file(r"geocod_parcial_corrigido.csv")
gdf.head()


,Protocolo,Endereço,Número,Bairro,Zona,Setor,DataInclusao,Cadastro,Assunto,Descrição,Origem,Cidade,Estado,Pais,DataInclusao_BR,endereco_completo,latitude,longitude,endereco_formatado
0,649554,Alameda Amor Perfeito,36,Zona 04 - centro,60,IAM - Fiscalização,2022-11-14 22:20:00,60130700,Poluição sonora por estabelecimento comercial,Som ao vivo fora do estabelecimento provocando...,Identificados,Maringá,Paraná,Brasil,22:20:00 14-11-2022,"Alameda Amor Perfeito, 36, Maringá - PR, Brasil",-23.3604883,-52.0148854,Alameda Amor Perfeito - Conj. Hab. Joao de Bar...
1,601042,Alameda Das Alfazemas,1968,Zona 3,60,IAM - Fiscalização,2022-05-21 23:54:00,60067500,Poluição sonora por estabelecimento comercial,Muito barulho som alto de dois bares thel bar ...,Identificados,Maringá,Paraná,Brasil,23:54:00 21-05-2022,"Alameda Das Alfazemas, 1968, Maringá - PR, Brasil",-23.3570352,-52.0095937,Alameda das Alfazemas - Conj. Hab. Joao de Bar...
2,660699,Alameda Doutor João Paulino,673,JARDIM ALVORADA I PARTE,24,IAM - Fiscalização,2022-12-23 23:49:00,24130900.0,Poluição sonora por estabelecimento comercial,Vizinho vários dias com som alto além da hora....,Sigilosos,Maringá,Paraná,Brasil,23:49:00 23-12-2022,"Alameda Doutor João Paulino, 673, Maringá - PR...",-23.4003971,-51.9212639,"Alameda Dr. João Paulino, 673 - Jardim Alvorad..."
3,784845,Avenida Advogado Horácio Raccanello Filho,183,Vila nova,18,IAM - Fiscalização,2023-12-23 08:56:00,18039700.0,Poluição sonora por estabelecimento comercial,Esse estabelecimento permanece com o som alto ...,Sigilosos,Maringá,Paraná,Brasil,08:56:00 23-12-2023,"Avenida Advogado Horácio Raccanello Filho, 183...",-23.4204466,-51.9249223,"Av. Advogado Horácio Raccanello Filho, 183 - Z..."
4,648166,Avenida Advogado Horácio Raccanello Filho,3605,ZONA ARMAZÉM,9,IAM - Fiscalização,2022-11-09 08:45:00,9014300.0,Poluição sonora por estabelecimento comercial,na madrugada de terça para quarta (dia 08 para...,Sigilosos,Maringá,Paraná,Brasil,08:45:00 09-11-2022,"Avenida Advogado Horácio Raccanello Filho, 360...",-23.4238947,-51.914974,"Av. Advogado Horácio Raccanello Filho, 3605 - ..."


In [39]:
from folium.plugins import HeatMapWithTime

# usa o df já carregado e com DataInclusao em datetime
df = df.sort_values("DataInclusao").copy()
df["latitude"] = df["latitude"].astype(float)
df["longitude"] = df["longitude"].astype(float)

# agrupar por dia (ajuste para outra granularidade se quiser: 'H' hora, etc.)
df["day"] = df["DataInclusao"].dt.floor("D")

# obter intervalo do slider (espera-se uma SelectionRangeSlider chamado `slider`)
try:
    s_val, e_val = slider.value
    start = pd.to_datetime(s_val)
    end = pd.to_datetime(e_val)
except Exception:
    # se não houver slider (ou ocorrer erro ao ler), usar todo o intervalo disponível
    start = df["day"].min()
    end = df["day"].max()
    # se não houver slider, usar todo o intervalo disponível
    start = df["day"].min()
    end = df["day"].max()

# filtrar o dataframe para o intervalo selecionado (inclusive)
df_window = df[(df["day"] >= start) & (df["day"] <= end)].copy()

# construir heat_data acumulado apenas dentro do intervalo do slider
heat_data = []
time_index = []
accum = []
if not df_window.empty:
    for day, g in df_window.groupby("day"):
        pts = g[["latitude", "longitude"]].values.tolist()
        accum.extend(pts)
        heat_data.append(accum.copy())  # cópia do acumulado até este dia (dentro do intervalo)
        time_index.append(day.strftime("%Y-%m-%d"))
else:
    # garantir variáveis válidas caso não haja dados no intervalo
    heat_data = []
    time_index = []

# criar mapa (centraliza na média dos pontos)
m = folium.Map(location=[df["latitude"].mean(), df["longitude"].mean()], zoom_start=12, tiles="OpenStreetMap")

# HeatMapWithTime sem o argumento `cumulative` (já fizemos a acumulação acima)
HeatMapWithTime(
    heat_data,
    index=time_index,
    auto_play=True,
    max_opacity=0.8,
    radius=15,
    gradient=None,
    use_local_extrema=False,
).add_to(m)

m.save("mapa_acumulado_heatmap2.html")

